In [162]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from os.path import expanduser
import re
from nltk.stem.porter import PorterStemmer

In [163]:
#pip install demoji
import demoji
#demoji.download_codes()

In [164]:
stop_words = [word.strip() for word in open('stop_words.txt').readlines()]

In [165]:
len(stop_words)


495

In [166]:
def stemming_tokenizer(str_input):
    porter_stemmer = PorterStemmer()
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [167]:
with open('dems.txt', 'r',encoding="utf-8") as file:
    dem_text = [line.strip('\n') for line in file]
with open('gop.txt', 'r',encoding="utf-8") as file:
    gop_text = [line.strip('\n') for line in file]
with open('NonPolitical.txt', 'r',encoding="utf-8") as file:
    nonp_text = [line.strip('\n') for line in file]

In [168]:
def clean_text(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text) #remove mentions
    text=re.sub(r'#[A-Za-z0-9]+','',text)  #removing hash tags
    text=re.sub(r'RT[\s]+','',text)
    text=re.sub(r'https?:\/\/\S+','',text)
    text=re.sub(r'[?|$|.|!|;|:|&|"|,|""]','',text)
    return text

In [211]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [212]:
#dem_text=dem_text.apply(clean_text)
dem_text = [clean_text(text) for text in dem_text]
gop_text = [clean_text(text) for text in gop_text]
nonp_text = [clean_text(text) for text in nonp_text]

In [213]:
#dem_text=dem_text.apply(clean_text)
dem_text = [remove_emoji(text) for text in dem_text]
gop_text = [remove_emoji(text) for text in gop_text]
nonp_text = [remove_emoji(text) for text in nonp_text]

In [215]:
vectorizer = CountVectorizer(input=dem_text + gop_text+nonp_text,
                             stop_words=stop_words,
                             max_features=1200,tokenizer=stemming_tokenizer)

In [216]:
dem_bow = vectorizer.fit_transform(dem_text)
gop_bow = vectorizer.fit_transform(gop_text)
nonp_bow = vectorizer.fit_transform(nonp_text)

C:\Users\pavan\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anybodi', 'anyon', 'anyth', 'anywher', 'avail', 'balanc', 'becam', 'becaus', 'becom', 'befor', 'certainli', 'clearli', 'coverag', 'doe', 'dure', 'earli', 'evenli', 'everi', 'everybodi', 'everyon', 'everyth', 'everywher', 'f796a80b0741d338640618cb3dc2547525884e6a', 'freeli', 'fulli', 'gener', 'ha', 'head', 'hi', 'howev', 'import', 'larg', 'mani', 'manual', 'mostli', 'necessari', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'perhap', 'possibl', 'quit', 'realli', 'sever', 'sinc', 'somebodi', 'someon', 'someth', 'somewher', 'therefor', 'thi', 'thu', 'togeth', 'veri', 'wa', 'whi'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [217]:
#%%
(dem_bow.shape, gop_bow.shape, nonp_bow.shape)
#%%

((3228, 1200), (3240, 1200), (12913, 1200))

In [218]:
x = sparse.vstack((dem_bow, gop_bow, nonp_bow))
ones = np.ones(dem_bow.shape[0])
zeros = np.zeros(gop_bow.shape[0])
twos = np.full(nonp_bow.shape[0],2)
y = np.hstack((ones, zeros, twos))


In [219]:
class_names = ['Democrats','Republicans','Non-Political']

In [220]:
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score  
from sklearn.neighbors import KNeighborsClassifier 
  
# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25,random_state = 42) 
  
# training a KNNClassifier 
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_test, y_test) 
print(accuracy)
  
# creating a confusion matrix 
knn_predictions = knn.predict(X_test)  
cm = confusion_matrix(y_test, knn_predictions) 


0.7769294263309946


In [221]:
#PLotting confusion matrix for decision tree classification
import matplotlib.pyplot as plt

from sklearn.metrics import plot_confusion_matrix

titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(knn, X_test, y_test,display_labels=class_names, cmap=plt.cm.Blues, normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (C:\Users\pavan\Anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [222]:
from sklearn.metrics import plot_confusion_matrix

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (C:\Users\pavan\Anaconda3\lib\site-packages\sklearn\metrics\__init__.py)